In [ ]:
!pip install -q torch==2.2.1 torchvision torchaudio -f https://download.pytorch.org/whl/torch_stable.html
!pip uninstall transformers -y
!pip install git+https://github.com/huggingface/transformers.git@main
!pip install -q ipywidgets
!pip install -q accelerate
!pip install -q jupyter
!pip install -q 'bitsandbytes-cuda113'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 GB 704.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.7/65.7 MB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 96.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!nvcc --version
!pip install -q bitsandbytes

import json, urllib
!gdown 19ekDpipQhBERlMLKVQaS6G6LGHp4WAlw

import bitsandbytes as bnb
bnb.__version__  # Ensures cuda is imported

# Check if CUDA is available in PyTorch
import torch
print("CUDA available:", torch.cuda.is_available())
import transformers
help(transformers)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.6 MB/s eta 0:00:00
Downloading...
From: https://drive.google.com/uc?id=19ekDpipQhBERlMLKVQaS6G6LGHp4WAlw
To: /content/training12b_yesno.json
100% 13.6M/13.6M [00:00<00:00, 95.2MB/s]


CUDA available: False


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/usr/local/lib/python3.10/dist-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [ ]:
import json
import torch
from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW, BertConfig, get_scheduler
from torch.utils.data import DataLoader, TensorDataset, RandomSampler
from sklearn.metrics import classification_report

def load_data(filepath):
    try:
        with open(filepath, 'r') as file:
            data = json.load(file)
        questions = data['questions']
        processed_questions = [q for q in questions if q['type'] == 'yesno']
        labels = [1 if q['exact_answer'].lower() == 'yes' else 0 for q in processed_questions]
        return processed_questions, labels
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        return None, None

def prepare_data(tokenizer, questions, labels, batch_size=16):
    bodies = [q['body'] for q in questions]
    input_encodings = tokenizer(bodies, padding=True, truncation=True, max_length=128, return_tensors="pt")
    inputs, masks = input_encodings['input_ids'], input_encodings['attention_mask']
    labels = torch.tensor(labels)
    dataset = TensorDataset(inputs, masks, labels, torch.arange(len(questions)))
    train_size = int(0.9 * len(dataset))
    val_size = len(dataset) - train_size
    train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])
    train_loader = DataLoader(train_dataset, sampler=RandomSampler(train_dataset), batch_size=batch_size)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    return train_loader, val_loader

def setup_model(model_id):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    config = BertConfig.from_pretrained(model_id, num_labels=2, hidden_dropout_prob=0.2, attention_probs_dropout_prob=0.2)
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForSequenceClassification.from_pretrained(model_id, config=config)
    model.to(device)
    return model, tokenizer, device

def train_model(model, device, train_loader, val_loader, epochs=10):
    class_weights = torch.tensor([1.6, 0.5], dtype=torch.float32).to(device)
    loss_function = nn.CrossEntropyLoss(weight=class_weights)
    optimizer = AdamW(model.parameters(), lr=5e-5)
    num_training_steps = epochs * len(train_loader)
    lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for batch in train_loader:
            inputs, masks, labels, _ = [b.to(device) for b in batch]
            model.zero_grad()
            outputs = model(inputs, attention_mask=masks, labels=labels)
            loss = loss_function(outputs.logits, labels)
            loss.backward()
            optimizer.step()
            lr_scheduler.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} -- Average Train Loss: {total_loss / len(train_loader)}")

    model.eval()
    all_predictions = []
    all_labels = []
    with torch.no_grad():
        for batch in val_loader:
            inputs, masks, labels, _ = [b.to(device) for b in batch]
            outputs = model(inputs, attention_mask=masks, labels=labels)
            predictions = torch.argmax(outputs.logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    print("Classification Report:")
    print(classification_report(all_labels, all_predictions, target_names=['No', 'Yes']))

def main():
    model_id = "bionlp/bluebert_pubmed_mimic_uncased_L-12_H-768_A-12"
    model, tokenizer, device = setup_model(model_id)
    questions, labels = load_data('/content/training12b_yesno_balanced.json')
    train_loader, val_loader = prepare_data(tokenizer, questions, labels)
    train_model(model, device, train_loader, val_loader)

if __name__ == "__main__":
    main()

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bionlp--bluebert_pubmed_mimic_uncased_L-12_H-768_A-12/snapshots/c656dbe1fc4d6c7771d93bcaff21b2e7984f64c8/config.json
Model config BertConfig {
  "attention_probs_dropout_prob": 0.2,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.2,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.43.0.dev0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bionlp--bluebert_pubmed_mimic_uncased_L-12_H-768_A-12/snapshots/c65

Epoch 1 -- Average Train Loss: 0.34800679726511474
Epoch 2 -- Average Train Loss: 0.2633003762622518
Epoch 3 -- Average Train Loss: 0.1953964103159942
Epoch 4 -- Average Train Loss: 0.12959927204615018
Epoch 5 -- Average Train Loss: 0.06297416410474914
Epoch 6 -- Average Train Loss: 0.04600126723901875
Epoch 7 -- Average Train Loss: 0.02128568155540696
Epoch 8 -- Average Train Loss: 0.027973913979958596
Epoch 9 -- Average Train Loss: 0.012745726483509193
Epoch 10 -- Average Train Loss: 0.015668672936653177
Classification Report:
              precision    recall  f1-score   support

          No       0.92      0.83      0.88       126
         Yes       0.81      0.91      0.86        99

    accuracy                           0.87       225
   macro avg       0.87      0.87      0.87       225
weighted avg       0.87      0.87      0.87       225

